# This file is based on https://github.com/dmlc/dgl/tree/master/examples/pytorch/hgt

In [12]:
#Import the right libraries
import pandas as pd
import dgl
import numpy as np
import networkx as nx
import torch
from dgl.data.utils import save_graphs
from dgl.data.utils import load_graphs
import sklearn.metrics as sk

In [13]:
#Import the data
processed_company_data = pd.read_csv('processed_company_data.csv', sep=",")
processed_investor_data = pd.read_csv('processed_investor_data.csv', sep=",")
processed_relation_data = pd.read_csv('processed_relation_data.csv', sep=",")

# Start of the github file

In [14]:
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
from hgt_model import *
import argparse

In [15]:
torch.manual_seed(0)

n_epoch = 200
n_hid = 256
n_inp = 256
clip = 1.0
max_lr = 1e-3

In [16]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [17]:
def train(model, G):
    best_val_acc = torch.tensor(0)
    best_test_acc = torch.tensor(0)
    
    best_fbeta = torch.tensor(0)
    best_precision = torch.tensor(0)
    best_recall = torch.tensor(0)
    
    
    train_step = torch.tensor(0)
    for epoch in np.arange(n_epoch) + 1:
        model.train()
        logits, _ = model(G, 'investor/company')
        # The loss is computed only for labeled nodes.
#         loss = F.cross_entropy(logits[train_idx], labels[train_idx].to(device))
        loss = F.cross_entropy(logits[train_idx], labels[train_idx])
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_step += 1
        scheduler.step(train_step)
        if epoch % 5 == 0:
            model.eval()
            logits, _ = model(G, 'investor/company')
            pred   = logits.argmax(1).cpu()
            train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
            val_acc   = (pred[val_idx]   == labels[val_idx]).float().mean()
            test_acc  = (pred[test_idx]  == labels[test_idx]).float().mean()
            
            fbeta = sk.fbeta_score(labels[test_idx], pred[test_idx], beta=0.2, average="macro")
            precision = sk.precision_score(labels[test_idx], pred[test_idx], average="macro")
            recall = sk.recall_score(labels[test_idx], pred[test_idx], average="macro")
            
            if best_val_acc < val_acc:
                best_val_acc = val_acc
                best_test_acc = test_acc
                
            if best_fbeta < fbeta:
                best_fbeta = fbeta
        
            if best_precision < precision:
                best_precision = precision
                
            if best_recall < recall:
                best_recall = recall
                
            print('Epoch: %d LR: %.5f Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
                epoch,
                optimizer.param_groups[0]['lr'], 
                loss.item(),
                train_acc.item(),
                val_acc.item(),
                best_val_acc.item(),
                test_acc.item(),
                best_test_acc.item()
            ))
            print('Epoch: %d LR: %.5f Loss %.4f, fbeta %.4f (Best %.4f), precision %.4f (Best %.4f), recall %.4f (Best %.4f)' % (
                epoch,
                optimizer.param_groups[0]['lr'], 
                loss.item(),
                fbeta.item(),
                best_fbeta.item(),
                precision.item(),
                best_precision.item(),
                recall.item(),
                best_recall.item()
            ))

In [18]:
g = load_graphs("dgl_graph")
G = g[0][0]
print(G)

Graph(num_nodes={'company': 9779, 'investor/company': 7883},
      num_edges={('company', 'different_invested_by', 'investor/company'): 50087, ('investor/company', 'different_invests_in', 'company'): 50087, ('investor/company', 'same_invested_by', 'investor/company'): 5102, ('investor/company', 'same_invests_in', 'investor/company'): 5102},
      metagraph=[('company', 'investor/company', 'different_invested_by'), ('investor/company', 'company', 'different_invests_in'), ('investor/company', 'investor/company', 'same_invested_by'), ('investor/company', 'investor/company', 'same_invests_in')])


In [19]:
different_edge = ('investor/company', 'different_invests_in', 'company')
same_edge = ('investor/company', 'same_invests_in', 'investor/company')

different_labels = G[different_edge].edges()
same_labels = G[same_edge].edges()

pid = G[different_edge].edges()[0]
labels = G[different_edge].edges()[1]

print(pid)
print(labels)

tensor([   0,    0,    0,  ..., 7882, 7882, 7882])
tensor([9181, 9701, 9676,  ..., 7817, 7818, 7819])


In [20]:
# generate train/val/test split
shuffle = np.random.permutation(pid)
train_idx = torch.tensor(shuffle[0:800]).long()
val_idx = torch.tensor(shuffle[800:900]).long()
test_idx = torch.tensor(shuffle[900:]).long()

In [21]:
node_dict = {}
edge_dict = {}

for ntype in G.ntypes:
    node_dict[ntype] = len(node_dict)
for etype in G.etypes:
    edge_dict[etype] = len(edge_dict)
    G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long) * edge_dict[etype] 

#     Random initialize input feature
for ntype in G.ntypes:
    emb = nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), 256), requires_grad = False)
    nn.init.xavier_uniform_(emb)
    G.nodes[ntype].data['inp'] = emb
    

# G = G.to(device)

In [22]:
model = HGT(G,
            node_dict, edge_dict,
            n_inp=n_inp,
            n_hid=n_hid,
            n_out=labels.max().item()+1,
            n_layers=2,
            n_heads=4,
            use_norm = True)#.to(device)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=n_epoch, max_lr = max_lr)
print('Training HGT with #param: %d' % (get_n_params(model)))
train(model, G)

Training HGT with #param: 3961687


/home/ramon/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/home/ramon/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ramon/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-def

Epoch: 5 LR: 0.00006 Loss 9.2663, Train Acc 0.0012, Val Acc 0.0000 (Best 0.0000), Test Acc 0.0002 (Best 0.0000)
Epoch: 5 LR: 0.00006 Loss 9.2663, fbeta 0.0000 (Best 0.0000), precision 0.0000 (Best 0.0000), recall 0.0005 (Best 0.0005)


/home/ramon/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/home/ramon/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 10 LR: 0.00011 Loss 9.1559, Train Acc 0.0012, Val Acc 0.0000 (Best 0.0000), Test Acc 0.0001 (Best 0.0000)
Epoch: 10 LR: 0.00011 Loss 9.1559, fbeta 0.0000 (Best 0.0000), precision 0.0000 (Best 0.0000), recall 0.0003 (Best 0.0005)


...

/home/ramon/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/home/ramon/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 195 LR: 0.00000 Loss 0.9006, Train Acc 0.9950, Val Acc 0.2700 (Best 0.2700), Test Acc 0.2856 (Best 0.2831)
Epoch: 195 LR: 0.00000 Loss 0.9006, fbeta 0.0738 (Best 0.1033), precision 0.0754 (Best 0.1108), recall 0.1120 (Best 0.1120)


/home/ramon/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 200 LR: 0.00000 Loss 0.9020, Train Acc 0.9950, Val Acc 0.2700 (Best 0.2700), Test Acc 0.2856 (Best 0.2831)
Epoch: 200 LR: 0.00000 Loss 0.9020, fbeta 0.0738 (Best 0.1033), precision 0.0754 (Best 0.1108), recall 0.1120 (Best 0.1120)


/home/ramon/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# torch.save({
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             }, "model.pth")

In [24]:
# model = HeteroRGCN(G,
#                    in_size=args.n_inp,
#                    hidden_size=args.n_hid,
#                    out_size=labels.max().item()+1)#.to(device)
# optimizer = torch.optim.AdamW(model.parameters())
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.n_epoch, max_lr = args.max_lr)
# print('Training RGCN with #param: %d' % (get_n_params(model)))
# train(model, G)

In [25]:
# model = HGT(G,
#             node_dict, edge_dict,
#             n_inp=args.n_inp,
#             n_hid=args.n_hid,
#             n_out=labels.max().item()+1,
#             n_layers=0,
#             n_heads=4)#.to(device)
# optimizer = torch.optim.AdamW(model.parameters())
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.n_epoch, max_lr = args.max_lr)
# print('Training MLP with #param: %d' % (get_n_params(model)))
# train(model, G)